<a href="https://colab.research.google.com/github/SkymanOne/COMP3217_CW2/blob/main/COMP3217_CW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PART A

Binary classification of logs.

## Naive approach
